In [4]:
import os
import re
import csv
import sys
import csv
import random
import ast

csv.field_size_limit(sys.maxsize)

131072

In [3]:
# load data
recs = []
with open("../Data/all-data.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                recs.append(row)
            
print len(recs)
recs[0]

39191


{'From': 'Kuwait',
 'action': "['establish']",
 'decision': 'support',
 'id': '0',
 'institutions': '[]',
 'notes': '',
 'text': '69.1 The establishment of an international round table to discuss the interrelation between security and fundamental freedoms (Kuwait). );',
 'theme': '[]',
 'to': 'Algeria',
 'year': '2008'}

## 1. Clean

In [3]:
# check for broken csv rows
for i in recs:
    if not i['year'].startswith('2'):
        print i['text']

In [33]:
# ordering
"""
numbs = []
for i in recs[:500]:
    n = i['text'].split()[0]
    n = n.split('.')
    i['n1'] = n[0]
    i['n2'] = n[1]
    if len(n) == 3:
        i['n3'] = n[2]
    else:
        i['n3'] = 0
"""

## 2. Sample

In [3]:
f = open('../sample-ids.txt','r')
sampled_ids = f.readlines()
f.close()
sampled_ids = [ast.literal_eval(i) for i in sampled_ids]
sampled_ids = [item for sublist in sampled_ids for item in sublist]

In [4]:
# remove already sampled recs
for i in recs:
    if i['id'] in sampled_ids:
        recs.remove(i)
len(recs)

38700

In [6]:
# sample
rec_ids = [int(i['id']) for i in recs] # get ids of remaining recs
ids = random.sample(rec_ids, 100) # sample those ids

sample = []
for i in recs:
    if int(i['id']) in ids:
        sample.append(i)
len(sample)

100

In [7]:
# delete data we don't want to include
for i in sample:
    del i['institutions']
    del i['theme']
    del i['action']
    del i['notes']
    
sample[0]

{'From': 'Portugal',
 'decision': 'support',
 'id': '433',
 'text': '56.21 To raise awareness regarding human rights and the rule of law (Portugal). );',
 'to': 'Mali',
 'year': '2008'}

In [8]:
#writing column headings
keys = sample[0].keys()

#writing the rest
with open('sample-erin-sentiment2.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(sample)

In [9]:
# add ids to the list of sampled data
ids = [str(i) for i in ids]
with open('../sample-ids.txt', 'a') as output_file:
    output_file.write("\n" + str(ids))

## 3. Test

In [89]:
# load data
recs = []
with open("../Data/all-data.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                recs.append(row)
            
len(recs)
recs[0]

{'From': 'Kuwait',
 'action': "['establish']",
 'decision': 'support',
 'id': '0',
 'institutions': '[]',
 'notes': '',
 'text': '69.1 The establishment of an international round table to discuss the interrelation between security and fundamental freedoms (Kuwait). );',
 'theme': '[]',
 'to': 'Algeria',
 'year': '2008'}

In [90]:
# load Erin
erin = []
with open("testing-3/erin-data.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                erin.append(row)
            
erin[1]

{'From': 'Holy See',
 'action': '[]',
 'decision': 'consider',
 'id': '1385',
 'institutions': '[]',
 'notes': '',
 'sentiment': '2',
 'text': '94.17 Investigate complaints concerning discrimination against members of minority religions, while developing educational and awareness programmes addressing these human rights violations (Holy See). );',
 'theme': "['discrimination', 'religion', 'education']",
 'to': 'Bangladesh',
 'year': '2009'}

In [91]:
# making literal lists
for i in erin:
    try:
        i['theme'] = ast.literal_eval(i['theme'])
        i['institutions'] = ast.literal_eval(i['institutions'])
        i['action'] = ast.literal_eval(i['action'])
    except Exception,e:
        print i
erin[1]

{'From': 'Holy See',
 'action': [],
 'decision': 'consider',
 'id': '1385',
 'institutions': [],
 'notes': '',
 'sentiment': '2',
 'text': '94.17 Investigate complaints concerning discrimination against members of minority religions, while developing educational and awareness programmes addressing these human rights violations (Holy See). );',
 'theme': ['discrimination', 'religion', 'education'],
 'to': 'Bangladesh',
 'year': '2009'}

In [92]:
# strip whitespace:
for i in erin:
    i['theme'] = [x.strip() for x in i['theme']]
    i['institutions'] = [x.strip() for x in i['institutions']]
    i['action'] = [x.strip() for x in i['action']]

In [93]:
# get ids
ids = [rec['id'] for rec in erin]

In [94]:
# get original sample
sample = []
for i in recs:
    if i['id'] in ids:
        sample.append(i)
sample[0]  

{'From': 'Malaysia',
 'action': '[]',
 'decision': 'consider',
 'id': '1247',
 'institutions': '[]',
 'notes': '',
 'text': '96.25 Step up its efforts to ensure that basic needs of the population are fully met, in particular, in the areas of employment, public health, education and public housing (Malaysia);',
 'theme': "['education', 'labor', 'health', 'infrastructure']",
 'to': 'Azerbaijan',
 'year': '2009'}

In [95]:
# make literal
for i in sample:
    i['theme'] = ast.literal_eval(i['theme'])
    i['institutions'] = ast.literal_eval(i['institutions'])
    i['action'] = ast.literal_eval(i['action'])

sample[1]

{'From': 'Holy See',
 'action': [],
 'decision': 'consider',
 'id': '1385',
 'institutions': [],
 'notes': '',
 'text': '94.17 Investigate complaints concerning discrimination against members of minority religions, while developing educational and awareness programmes addressing these human rights violations (Holy See). );',
 'theme': ['education', 'discrimination', 'religion', 'ethnic'],
 'to': 'Bangladesh',
 'year': '2009'}

In [96]:
# zip
matching = []
for i in range(100):
    dic = erin[i].copy()
    
    # Theme
    dic['theme-erin'] = erin[i]['theme']
    dic['theme-rochelle'] = sample[i]['theme']
    del dic['theme']
    
    if set(dic['theme-erin']) == set(dic['theme-rochelle']):
        dic['theme-match'] = True
    elif set(dic['theme-erin']).intersection(dic['theme-rochelle']):
        dic['theme-match'] = "Partial"
    else:
        dic['theme-match'] = False
    
    # institutions
    dic['institution-erin'] = erin[i]['institutions']
    dic['institution-rochelle'] = sample[i]['institutions']
    del dic['institutions']
    
    if set(dic['institution-erin']) == set(dic['institution-rochelle']):
        dic['institution-match'] = True
    elif set(dic['institution-erin']).intersection(dic['institution-rochelle']):
        dic['institution-match'] = "Partial"
    else:
        dic['institution-match'] = False
    
    # actions
    dic['action-erin'] = erin[i]['action']
    dic['action-rochelle'] = sample[i]['action']
    del dic['action']
    
    if set(dic['action-erin']) == set(dic['action-rochelle']):
        dic['action-match'] = True
    elif set(dic['action-erin']).intersection(dic['action-rochelle']):
        dic['action-match'] = "Partial"
    else:
        dic['action-match'] = False
    
    matching.append(dic)

In [97]:
# make lists of matches, etc
theme_match = [i for i in matching if i['theme-match'] == True]
theme_partial = [i for i in matching if i['theme-match'] == "Partial"]
theme_false = [i for i in matching if i['theme-match'] == False]

institution_match = [i for i in matching if i['institution-match'] == True]
institution_partial = [i for i in matching if i['institution-match'] == "Partial"]
institution_false = [i for i in matching if i['institution-match'] == False]

action_match = [i for i in matching if i['action-match'] == True]
action_partial = [i for i in matching if i['action-match'] == "Partial"]
action_false = [i for i in matching if i['action-match'] == False]

In [98]:
print "Theme" + "\n"
print "Match: " + str(len(theme_match))
print "Partial: " + str(len(theme_partial))
print "False: " + str(len(theme_false))
print "\n"

print "Institutions" + "\n"
print "Match: " + str(len(institution_match))
print "Partial: " + str(len(institution_partial))
print "False: " + str(len(institution_false))
print "\n"

print "Action" + "\n"
print "Match: " + str(len(action_match))
print "Partial: " + str(len(action_partial))
print "False: " + str(len(action_false))
print "\n"

Theme

Match: 80
Partial: 16
False: 4


Institutions

Match: 93
Partial: 1
False: 6


Action

Match: 98
Partial: 1
False: 1




In [105]:
for i in action_partial:
    print i['id'] + ": " + i['text']
    print "ERIN: " + str(i['action-erin'])
    print "ROCHELLE: " + str(i['action-rochelle']) + "\n"

13708: 77.40. Take several measures in the area of the administration of justice to accelerate the reform of the judicial system, to fight against abuses of preventive detention, and to draft and transmit the report expected since 2005 by the Committee against Torture (France);
ERIN: ['take-measures', 'report']
ROCHELLE: ['take-measures']



## 4. Turn to Binary SpreadSheet

In [106]:
institutions = ['core', 'CRPD', 'ICCPR', 'eu-vaw', 'geneva', 'ICC', 'ILO', 'ICESR', 'achr', 'CAT', 'CRC', 'echr', '1954-stateless', 'traffick', 'CTOC', 'ICMW', 'ICERD', 'eu-child', 'sp-pr', 'PP', 'kampala', '1951refugees', 'pol-women', 'CPED', 'uncac', 'ottawa', '1961-stateless', 'CEDAW', 'unesco-ed', 'hague', 'genocide', 'HRC', 'unesco-expr', 'int-cord', 'frame-minorities', 'ind169', 'OP', 'ic-mercenaries', 'eu-migrant', 'un-indig', 'vienna', 'ouagadougou', 'UNHCR']

In [107]:
actions = ['accede', 'ratify', 'take-measures', 'sign', 'report', 'continue', 'withdraw', 'harmonize', 'party', 'implement', 'establish']

In [108]:
theme = ['death-pen', 'infrastructure ', 'intelligence', 'prisoners', 'elderly', 'culture', 'education', 'children', 'civil-so', 'disappearances', 'police', 'hiv', 'privacy', 'discrimination', 'impunity', 'governance', 'labor', 'environment', 'religion', 'foreign-movement', 'health', 'speech', 'domest-movement', 'terrorism', 'paramilitary', 'corruption', 'sex-violence', 'civil-conflict', 'race', 'torture', 'food', 'extra-judicial', 'judiciary', 'indigenous', 'lgbt', 'self-determ', 'poverty', 'ethnic', 'women', 'land', 'reproductive', 'post-conflict', 'disabilities', 'migrants', 'pol-imprisonment', 'socialism', 'trafficking', 'military', 'property']

In [109]:
erin[0].keys()

['From',
 'sentiment',
 'text',
 'decision',
 'id',
 'to',
 'theme',
 'notes',
 'year',
 'action',
 'institutions']

In [110]:
for i in sample:
    for s in institutions:
        if s in i['institutions']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in actions:
        if s in i['action']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in theme:
        if s in i['theme']:
            i[s] = 1
        else:
            i[s] = 0 
    
    del i['to']
    del i['From']
    del i['text']
    del i['notes']
    del i['action']
    del i['theme']
    del i['decision']
    del i['institutions']
    del i['year']

In [111]:
for i in erin:
    for s in institutions:
        if s in i['institutions']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in actions:
        if s in i['action']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in theme:
        if s in i['theme']:
            i[s] = 1
        else:
            i[s] = 0  
            
    del i['to']
    del i['From']
    del i['text']
    del i['year']
    del i['institutions']
    del i['action']
    del i['theme']
    del i['decision']
    del i['sentiment']

In [112]:
## writing sample

#writing column headings
keys = sample[0].keys()

#writing the rest
with open('testing-3/sample-binary.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(sample)

In [113]:
## writing erin

#writing column headings
keys = erin[0].keys()

#writing the rest
with open('testing-3/erin-binary.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(erin)

## 5. Binary - all

In [145]:
# load data
recs = []
with open("../Data/all-data.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                recs.append(row)
            
len(recs)
recs[0]

{'From': 'Kuwait',
 'action': "['establish']",
 'decision': 'support',
 'id': '0',
 'institutions': '[]',
 'notes': '',
 'text': '69.1 The establishment of an international round table to discuss the interrelation between security and fundamental freedoms (Kuwait). );',
 'theme': '[]',
 'to': 'Algeria',
 'year': '2008'}

In [146]:
# make literal
for i in recs:
    i['theme'] = ast.literal_eval(i['theme'])
    i['institutions'] = ast.literal_eval(i['institutions'])
    i['action'] = ast.literal_eval(i['action'])

recs[1]

{'From': 'Sudan',
 'action': [],
 'decision': 'support',
 'id': '1',
 'institutions': [],
 'notes': '',
 'text': '69.2 That Algeria share its experience in the fight against terrorism and pursue its efforts to combat poverty and promote economic, social and cultural rights with a view to achieving the Millennium Development Goals by 2015 (Sudan). );',
 'theme': ['culture', 'terrorism', 'poverty'],
 'to': 'Algeria',
 'year': '2008'}

In [147]:
# make dummies
for i in recs:
    for s in institutions:
        if s in i['institutions']:
            i[s] = 1
        else:
            i[s] = 0

    for s in actions:
        if s in i['action']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in theme:
        if s in i['theme']:
            i[s] = 1
        else:
            i[s] = 0  

In [148]:
# lists to strings
for i in recs:
    if i['theme']:
        i['theme'] = ", ".join(i['theme'])
    if i['action']:
        i['action'] = ", ".join(i['action'])
    if i['institutions']:
        i['institutions'] = ", ".join(i['institutions'])

In [149]:
recs[1]['theme']

'culture, terrorism, poverty'

In [150]:
## write

#writing column headings
keys = recs[0].keys()

#writing the rest
with open('../Data/all-binary.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(recs)